## Introduction

Data loading into a SCD table involves a first-time bulk data loading, referred to as the _initial data load_. This is followed by continuous or regular data loading, referred to as an _incremental data load_, to keep the records up to date with changes in the source tables.

To demonstrate the solution, we walk through the following steps for initial data load (1–7) and incremental data load (8–12):

1. Land the source data files in an Amazon S3 location, using one subfolder per source table.
2. Use an [AWS Glue](https://aws.amazon.com/glue) crawler to parse the data files and register tables in the AWS Glue Data Catalog.
3. Create an external schema in Amazon Redshift to point to the AWS Glue database containing these tables.
4. In Amazon Redshift, create one view per source table to fetch the latest version of the record for each primary key (`customer_id`) value.
5. Create the `dim_customer` table in Amazon Redshift, which contains attributes from all relevant source tables.
6. Create a view in Amazon Redshift joining the source table views from Step 4 to project the attributes modeled in the dimension table.
7. Populate the initial data from the view created in Step 6 into the `dim_customer` table, generating `customer_sk`.
8. Land the incremental data files for each source table in their respective Amazon S3 location.
9. In Amazon Redshift, create a temporary table to accommodate the change-only records.
10. Join the view from Step 6 and `dim_customer` and identify change records comparing the combined hash value of attributes. Populate the change records into the temporary table with an `I`, `U`, or `D` indicator.
11. Update `rec_exp_dt` in `dim_customer` for all `U` and `D` records from the temporary table.
12. Insert records into `dim_customer`, querying all `I` and `U` records from the temporary table.

## Process Flow

![](./img.drawio.svg)

## Setup

In [15]:
import boto3
import json

def get_secret(secret_name, region_name="us-east-1"):
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name)
    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    get_secret_value_response = json.loads(get_secret_value_response['SecretString'])
    return get_secret_value_response

creds = get_secret("wysde")
USERNAME = creds["REDSHIFT_USERNAME"]
PASSWORD = creds["REDSHIFT_PASSWORD"]
HOST = creds["REDSHIFT_HOST"]
PORT = 5439
DATABASE = 'dev'

conn_str = 'postgresql://{0}:{1}@{2}:{3}/{4}'.format(USERNAME, PASSWORD, HOST, PORT, DATABASE)

%config SqlMagic.autopandas=True
%config SqlMagic.displaycon=False
%config SqlMagic.feedback=False
%config SqlMagic.displaylimit=5
%reload_ext sql
%sql {conn_str}

## Initial Data Load

### Land data from source tables

In [ ]:
#download the data from source
! wget -q --show-progress https://github.com/datalaker/assets/files/11010509/customer_address_with_ts.csv -O data/customer_address_with_ts.csv
! wget -q --show-progress https://github.com/datalaker/assets/files/11010510/customer_master_with_ts.csv -O data/customer_master_with_ts.csv

In [2]:
#load into s3
! aws s3 cp data/customer_address_with_ts.csv s3://wysde-datasets/customers/customer_address/customer_address_with_ts.csv
! aws s3 cp data/customer_master_with_ts.csv s3://wysde-datasets/customers/customer_master/customer_master_with_ts.csv

upload: ./customer_address_with_ts.csv to s3://wysde-datasets/customers/customer_address/customer_address_with_ts.csv
upload: ./customer_master_with_ts.csv to s3://wysde-datasets/customers/customer_master/customer_master_with_ts.csv


### Register source tables in the AWS Glue Data Catalog

In [9]:
#create crawler    
!aws glue create-crawler \
    --name rs-dimension \
    --role service-role/AWSGlueServiceRole-FullS3Access \
    --database-name datalake \
    --targets file://glue-targets.json

In [10]:
#run crawler
!aws glue start-crawler \
    --name rs-dimension

In [14]:
#check status
!aws glue get-crawler \
    --name rs-dimension \
    --query "Crawler.LastCrawl"

{
    "Status": "SUCCEEDED",
    "LogGroup": "/aws-glue/crawlers",
    "LogStream": "rs-dimension",
    "MessagePrefix": "fc9ed2c2-0937-4f04-bdfb-57ae9aabf73d",
    "StartTime": "2023-03-19T14:21:58+05:30"
}


### Create schemas in Amazon Redshift

In [16]:
%%sql

create external schema spectrum_dim
from data catalog database 'datalake' iam_role default;

""


Check if the tables registered in the Data Catalog in the preceding section are visible from within Amazon Redshift:

In [17]:
%%sql

select *
from spectrum_dim.customer_master
limit 10;

,customer_id,first_name,last_name,employer_name,row_audit_ts
0,1,Gladys,Rim,RIM Inc.,2019-05-03 14:52:31
1,2,Yuki,Whobrey,Reims Collectables,2019-05-03 14:52:32
2,3,Fletcher,Flosi,Lyon Souveniers,2019-05-03 14:52:33
3,4,Bette,Nicka,Toys4GrownUps.com,2019-05-03 14:52:34
4,5,Vinouye,Foller,Corporate Gift Ideas Co.,2019-05-03 14:52:35


In [18]:
%%sql

select *
from spectrum_dim.customer_address
limit 10;

,customer_id,email_id,city,country,row_audit_ts
0,1,gladys.rim@rim.org,NYC,USA,2019-05-03 14:52:36
1,2,yuki_whobrey@aol.com,Reims,France,2019-05-03 14:52:37
2,3,fletcher.flosi@yahoo.com,Paris,France,2019-05-03 14:52:38
3,4,bette_nicka@cox.net,Pasadena,USA,2019-05-03 14:52:39
4,5,vinouye@aol.com,San Francisco,USA,2019-05-03 14:52:40


Create another schema in Amazon Redshift to host the table, dim_customer:

In [19]:
%%sql

create schema rs_dim;

""


### Create views to fetch the latest records from each source table

Create a view for the customer_master table, naming it `vw_cust_mstr_latest`:

In [21]:
%%sql

create view rs_dim.vw_cust_mstr_latest as with rows_numbered as (
    select customer_id,
        first_name,
        last_name,
        employer_name,
        row_audit_ts,
        row_number() over(
            partition by customer_id
            order by row_audit_ts desc
        ) as rnum
    from spectrum_dim.customer_master
)
select customer_id,
    first_name,
    last_name,
    employer_name,
    row_audit_ts,
    rnum
from rows_numbered
where rnum = 1 with no schema binding;

""


The preceding query uses [row\_number](https://docs.aws.amazon.com/redshift/latest/dg/r_WF_ROW_NUMBER.html), which is a window function provided by Amazon Redshift. Using window functions enables you to create analytic business queries more efficiently. Window functions operate on a partition of a result set, and return a value for every row in that window. The `row_number` window function determines the ordinal number of the current row within a group of rows, counting from 1, based on the ORDER BY expression in the OVER clause. By including the PARTITION BY clause as `customer_id`, groups are created for each value of `customer_id` and ordinal numbers are reset for each group.

Create a view for the `customer_address` table, naming it `vw_cust_addr_latest`:

In [22]:
%%sql

create view rs_dim.vw_cust_addr_latest as with rows_numbered as (
    select customer_id,
        email_id,
        city,
        country,
        row_audit_ts,
        row_number() over(
            partition by customer_id
            order by row_audit_ts desc
        ) as rnum
    from spectrum_dim.customer_address
)
select customer_id,
    email_id,
    city,
    country,
    row_audit_ts,
    rnum
from rows_numbered
where rnum = 1 with no schema binding;

""


Both view definitions use the `row_number` window function of Amazon Redshift, ordering the records by descending order of the `row_audit_ts` column (the audit timestamp column). The condition `rnum=1` fetches the latest record for each `customer_id` value.

### Create the dim\_customer table in Amazon Redshift

Create `dim_customer` as an internal table in Amazon Redshift within the `rs_dim` schema. The dimension table includes the column `customer_sk`, that acts as the surrogate key column and enables us to capture a time-sensitive version of each customer record. The validity period for each record is defined by the columns `rec_eff_dt` and `rec_exp_dt`, representing record effective date and record expiry date, respectively. See the following code:

In [23]:
%%sql

create table rs_dim.dim_customer (
  customer_sk bigint, 
  customer_id bigint, 
  first_name varchar(100), 
  last_name varchar(100), 
  employer_name varchar(100), 
  email_id varchar(100), 
  city varchar(100), 
  country varchar(100), 
  rec_eff_dt date, 
  rec_exp_dt date
) diststyle auto;

""


### Create a view to consolidate the latest version of source records

Create the view `vw_dim_customer_src`, which consolidates the latest records from both source tables using `left outer join`, keeping them ready to be populated into the Amazon Redshift dimension table. This view fetches data from the latest views defined in the section “Create views to fetch the latest records from each source table”:

In [24]:
%%sql

create view rs_dim.vw_dim_customer_src as
select m.customer_id,
    m.first_name,
    m.last_name,
    m.employer_name,
    a.email_id,
    a.city,
    a.country
from rs_dim.vw_cust_mstr_latest as m
    left join rs_dim.vw_cust_addr_latest as a on m.customer_id = a.customer_id
order by m.customer_id with no schema binding;

""


At this point, this view fetches the initial data for loading into the `dim_customer` table that we are about to create. In your use-case, use a similar approach to create and join the required source table views to populate your target dimension table.

### Populate initial data into dim\_customer

Populate the initial data into the `dim_customer` table by querying the view `vw_dim_customer_src`. Because this is the initial data load, running row numbers generated by the `row_number` window function will suffice to populate a unique value in the `customer_sk` column starting from 1:

In [25]:
%%sql

insert into rs_dim.dim_customer
select row_number() over() as customer_sk,
    customer_id,
    first_name,
    last_name,
    employer_name,
    email_id,
    city,
    country,
    cast('2022-07-01' as date) rec_eff_dt,
    cast('9999-12-31' as date) rec_exp_dt
from rs_dim.vw_dim_customer_src;

""


In this query, we have specified `’2022-07-01’` as the value in `rec_eff_dt` for all initial data records. For your use-case, you can modify this date value as appropriate to your situation.

The preceding steps complete the initial data loading into the `dim_customer` table. In the next steps, we proceed with populating incremental data.

## Incremental Data Load

### Land ongoing change data files in Amazon S3

After the initial load, the source systems provide data files on an ongoing basis, either containing only new and change records or a full extract containing all records for a particular table.

In [ ]:
#download the data from source
! wget -q --show-progress https://github.com/datalaker/assets/files/11010580/customer_address_with_ts_incr.csv -O data/customer_address_with_ts_incr.csv
! wget -q --show-progress https://github.com/datalaker/assets/files/11010581/customer_master_with_ts_incr.csv -O data/customer_master_with_ts_incr.csv

In [27]:
#load into s3
! aws s3 cp data/customer_address_with_ts_incr.csv s3://wysde-datasets/customers/customer_address/customer_address_with_ts_incr.csv
! aws s3 cp data/customer_master_with_ts_incr.csv s3://wysde-datasets/customers/customer_master/customer_master_with_ts_incr.csv

upload: data/customer_address_with_ts_incr.csv to s3://wysde-datasets/customers/customer_address/customer_address_with_ts_incr.csv
upload: data/customer_master_with_ts_incr.csv to s3://wysde-datasets/customers/customer_master/customer_master_with_ts_incr.csv


For this example, we have uploaded the incremental data for the `customer_master` and `customer_address` source tables with a few `customer_id` records receiving updates and a few new records being added.

### Create a temporary table to capture change records

Create the temporary table `temp_dim_customer` to store all changes that need to be applied to the target `dim_customer` table:

In [28]:
%%sql

create temp table temp_dim_customer (
    customer_sk bigint,
    customer_id bigint,
    first_name varchar(100),
    last_name varchar(100),
    employer_name varchar(100),
    email_id varchar(100),
    city varchar(100),
    country varchar(100),
    rec_eff_dt date,
    rec_exp_dt date,
    iud_operation character(1)
);

""


### Populate the temporary table with new and changed records

This is a multi-step process that can be combined into a single complex SQL. Complete the following steps:

1. Fetch the latest version of all customer attributes by querying the view `vw_dim_customer_src`.

Amazon Redshift offers hashing functions such as [sha2](https://docs.aws.amazon.com/redshift/latest/dg/SHA2.html), which converts a variable length string input into a fixed length character output. The output string is a text representation of the hexadecimal value of the checksum with the specified number of bits. In this case, we pass a concatenated set of customer attributes whose change we want to track, specifying the number of bits as 512. We’ll use the output of the hash function to determine if any of the attributes have undergone a change. This dataset will be called `newver` (new version).

Because we landed the ongoing change data in the same location as the initial data files, the records retrieved from the preceding query (in `newver`) include all records, even the unchanged ones. But because of the definition of the view `vw_dim_customer_src`, we get only one record per customerid, which is its latest version based on `row_audit_ts`.

2. In a similar manner, retrieve the latest version of all customer records from `dim_customer`, which are identified by `rec_exp_dt=‘9999-12-31’`. While doing so, also retrieve the `sha2` value of all customer attributes available in `dim_customer`. This dataset will be called oldver (old or existing version).
3. Identify the current maximum surrogate key value from the `dim_customer` table.

This value (maxval) will be added to the row_number before being used as the customer_sk value for the change records that need to be inserted.

4. Perform a full outer join of the old version of records (`oldver`) and the new version (`newver`) of records on the `customer_id` column. Then compare the old and new hash values generated by the `sha2` function to determine if the change record is an insert, update, or delete.

We tag the records as follows:

- If the `customer_id` is non-existent in the `oldver` dataset (`oldver.customer_id is null`), it’s tagged as an insert (`‘I'`).
- Otherwise, if the `customer_id` is non-existent in the `newver` dataset (`newver.customer_id is null`), it’s tagged as a delete (`‘D'`).
- Otherwise, if the old `hash_value` and new `hash_value` are different, these records represent an update (`‘U'`).
- Otherwise, it indicates that the record has not undergone any change and therefore can be ignored or marked as not-to-be-processed (`‘N'`).

Make sure to modify the preceding logic if the source extract contains `rec_source_status` to identify deleted records.

Although `sha2` output maps a possibly infinite set of input strings to a finite set of output strings, the chances of collision of hash values for the original row values and changed row values are very unlikely. Instead of individually comparing each column value before and after, we compare the hash values generated by `sha2` to conclude if there has been a change in any of the attributes of the customer record. For your use-case, we recommend you choose a [hash function](https://docs.aws.amazon.com/redshift/latest/dg/hash-functions.html) that works for your data conditions after adequate testing. Instead, you can compare individual column values if none of the hash functions satisfactorily meet your expectations.

5. Combining the outputs from the preceding steps, let’s create the INSERT statement that captures only change records to populate the temporary table.

> Since we are receiving incremental data from source we will receive only I's, U's In case D records are also received, it is expected that there will be a STATUS column with INACTIVE as value Which will be treated as U and existing latest record will be expired. In case of full refresh, the below query will also retrieve D type records against all PK values which are absent in the latest snapshot.

Populate all new inserts and updates to the temp_dim_customer table by using query below

In [30]:
%%sql

insert into temp_dim_customer (
        customer_sk,
        customer_id,
        first_name,
        last_name,
        employer_name,
        email_id,
        city,
        country,
        rec_eff_dt,
        rec_exp_dt,
        iud_operation
    ) with newt as (
        select customer_id,
            sha2(
                coalesce(first_name, '') || coalesce(last_name, '') || coalesce(employer_name, '') || coalesce(email_id, '') || coalesce(city, '') || coalesce(country, ''),
                512
            ) as hash_value,
            first_name,
            last_name,
            employer_name,
            email_id,
            city,
            country,
            current_date rec_eff_dt,
            cast('9999-12-31' as date) rec_exp_dt
        from rs_dim.vw_dim_customer_src
    ),
    oldt as (
        select customer_id,
            sha2(
                coalesce(first_name, '') || coalesce(last_name, '') || coalesce(employer_name, '') || coalesce(email_id, '') || coalesce(city, '') || coalesce(country, ''),
                512
            ) as hash_value,
            first_name,
            last_name,
            employer_name,
            email_id,
            city,
            country
        from rs_dim.dim_customer
        where rec_exp_dt = '9999-12-31'
    ),
    maxsk as (
        select max(customer_sk) as maxval
        from rs_dim.dim_customer
    ),
    allrecs as (
        select coalesce(oldt.customer_id, newt.customer_id) as customer_id,
            case
                when oldt.customer_id is null then 'I'
                when newt.customer_id is null then 'D'
                when oldt.hash_value != newt.hash_value then 'U'
                else 'N'
            end as iud_op,
            newt.first_name,
            newt.last_name,
            newt.employer_name,
            newt.email_id,
            newt.city,
            newt.country,
            newt.rec_eff_dt,
            newt.rec_exp_dt
        from oldt
            full outer join newt on oldt.customer_id = newt.customer_id
    )
select (maxval +(row_number() over())) as customer_sk,
    customer_id,
    first_name,
    last_name,
    employer_name,
    email_id,
    city,
    country,
    rec_eff_dt,
    rec_exp_dt,
    iud_op
from allrecs,
    maxsk
where iud_op != 'N';

""


### Expire updated customer records

With the `temp_dim_customer` table now containing only the change records (either `‘I’`, `‘U’`, or `‘D’`), the same can be applied on the target `dim_customer` table.

Let’s first fetch all records with values `‘U’` or `‘D’` in the `iud_op` column. These are records that have either been deleted or updated in the source system. Because `dim_customer` is a slowly changing dimension, it needs to reflect the validity period of each customer record. In this case, we expire the presently active records that have been updated or deleted. We expire these records as of yesterday (by setting `rec_exp_dt=current_date-1`) matching on the `customer_id` column:

In [32]:
%%sql

update rs_dim.dim_customer
set rec_exp_dt = current_date -1
where customer_id in (
        select customer_id
        from temp_dim_customer as t
        where iud_operation in ('U', 'D')
    )
    and rec_exp_dt = '9999-12-31';

""


### Insert new and changed records

As the last step, we need to insert the newer version of updated records along with all first-time inserts. These are indicated by `‘U’` and `‘I’`, respectively, in the `iud_op` column in the `temp_dim_customer` table:

In [33]:
%%sql

insert into rs_dim.dim_customer (
        customer_sk,
        customer_id,
        first_name,
        last_name,
        employer_name,
        email_id,
        city,
        country,
        rec_eff_dt,
        rec_exp_dt
    )
select customer_sk,
    customer_id,
    first_name,
    last_name,
    employer_name,
    email_id,
    city,
    country,
    rec_eff_dt,
    rec_exp_dt
from temp_dim_customer
where iud_operation in ('I', 'U');

""


Depending on the SQL client setting, you might want to run a `commit transaction;` command to verify that the preceding changes are persisted successfully in Amazon Redshift.

In [34]:
%sql commit;

""


### Check the final output

You can run the following query and see that the `dim_customer` table now contains both the initial data records plus the incremental data records, capturing multiple versions for those `customer_id` values that got changed as part of incremental data loading. The output also indicates that each record has been populated with appropriate values in `rec_eff_dt` and `rec_exp_dt` corresponding to the record validity period.

In [36]:
%%sql

select *
from rs_dim.dim_customer
order by customer_id,
    customer_sk;

,customer_sk,customer_id,first_name,last_name,employer_name,email_id,city,country,rec_eff_dt,rec_exp_dt
0,1,1,Gladys,Rim,RIM Inc.,gladys.rim@rim.org,NYC,USA,2022-07-01,9999-12-31
1,5,2,Yuki,Whobrey,Reims Collectables,yuki_whobrey@aol.com,Reims,France,2022-07-01,2023-03-18
2,9,2,Yuki,Whobrey,Collectica Inc,yuki_whobrey@collectica.com,Reims,France,2023-03-19,9999-12-31
3,3,3,Fletcher,Flosi,Lyon Souveniers,fletcher.flosi@yahoo.com,Paris,France,2022-07-01,9999-12-31
4,2,4,Bette,Nicka,Toys4GrownUps.com,bette_nicka@cox.net,Pasadena,USA,2022-07-01,2023-03-18
5,6,4,Bette,Nicka,Souveniers Inc,bette_nicka@souveniers.com,Pasadena,USA,2023-03-19,9999-12-31
6,4,5,Vinouye,Foller,Corporate Gift Ideas Co.,vinouye@aol.com,San Francisco,USA,2022-07-01,9999-12-31
7,7,101,Arun,Raman,Land of Toys Inc.,arun.raman@lotoys.org,NYC,USA,2023-03-19,9999-12-31
8,8,102,Ganapathy,Krish,Amazing Toys,krish.ganapathy@amazingtoys.com,Reims,France,2023-03-19,9999-12-31
9,10,103,William,Butt,Toyzomaniac Inc.,william.butt@yahoo.com,Paris,France,2023-03-19,9999-12-31


## Conclusion

In this lab, you learned how to simplify data loading into Type-2 SCD tables in Amazon Redshift, covering both initial data loading and incremental data loading. The approach deals with multiple source tables populating a target dimension table, capturing the latest version of source records as of each run.